In [1]:
import os
import sys

import pandas as pd
import numpy as np

import networkx as nx

from basicmemnet import memnet

from sklearn.metrics import confusion_matrix

In [2]:
md_train = memnet.DSL()
md_test = memnet.DSL()

md_train.import_gml("train_graph.gml")
md_test.import_gml("test_graph.gml")

test = md_test.get_graph()
train = md_train.get_graph()

In [3]:
parent_actions = ['task_1_k_cooking', 'task_2_k_cooking_with_bowls', 'task_3_k_pouring', 'task_4_k_wiping',
                  'task_5_k_cereals', 'task_6_w_hard_drive', 'task_7_w_free_hard_drive', 'task_8_w_hammering', 
                  'task_9_w_sawing']

In [10]:
for a, b, link in test.out_edges('668f64fb8cc61c379809bfee', data=True):
    print(test.nodes[a].get('utterances')[0], link['link_type'], test.nodes[b].get('utterances')[0])

stir has_object whisk
stir has_next place


### Create Dictionary for Rule-based Baseline

In [12]:
dict = {}
for u, v, data in train.edges(data=True):
    relation = data.get('link_type')

    if relation == 'has_element':
        head = train.nodes[u].get('utterances')[0] # head is parent action (e.g. cooking, cereals...)
        tail = train.nodes[v].get('utterances')[0] # tail is sub-action (e.g. approach, lift...)

        # If tail action has object, keep it. 
        for a, b, link in train.out_edges(v, data=True):
            if link['link_type'] == 'has_object':
                tail = tail + ' ' + train.nodes[b].get('utterances')[0]
                # As a result, tail will look like: stir whisk, cut knife, etc...

        if tail not in dict.keys():
            dict[tail] = {}

        if head not in dict[tail].keys():
            dict[tail][head] = 1
        else:
            dict[tail][head] += 1

In [14]:
baseline_dict = {}

for action, children in dict.items():
    # Find the child node with the maximum count
    max_child = max(children, key=children.get)
    baseline_dict[action] = max_child

In [16]:
baseline_dict

{'idle': 'task_8_w_hammering',
 'approach whisk': 'task_1_k_cooking',
 'lift whisk': 'task_1_k_cooking',
 'stir whisk': 'task_1_k_cooking',
 'place whisk': 'task_1_k_cooking',
 'retreat whisk': 'task_1_k_cooking',
 'approach bowl': 'task_2_k_cooking_with_bowls',
 'hold bowl': 'task_1_k_cooking',
 'retreat bowl': 'task_2_k_cooking_with_bowls',
 'approach bottle': 'task_3_k_pouring',
 'lift bottle': 'task_3_k_pouring',
 'pour bottle': 'task_3_k_pouring',
 'hold bottle': 'task_4_k_wiping',
 'place bottle': 'task_3_k_pouring',
 'retreat bottle': 'task_3_k_pouring',
 'approach cereals': 'task_5_k_cereals',
 'lift cereals': 'task_5_k_cereals',
 'pour cereals': 'task_5_k_cereals',
 'place cereals': 'task_5_k_cereals',
 'retreat cereals': 'task_5_k_cereals',
 'lift bowl': 'task_2_k_cooking_with_bowls',
 'pour bowl': 'task_2_k_cooking_with_bowls',
 'place bowl': 'task_2_k_cooking_with_bowls',
 'idle whisk': 'task_2_k_cooking_with_bowls',
 'retreat': 'task_2_k_cooking_with_bowls',
 'idle bowl': 

### Generate Predictions by Baseline Dictionary

In [18]:
real_utterances = []  # True values
predictions = []  # Top 1 predictions
predictions_probab = []  # All predictions - so this one will be the list of dictionaries, 
                         # with parent actions as keys, number of votes for that action as values.

for node in test.nodes(data=True):
    node_id = node[0]   # String, e.g. "658190c06eccd77ab5dc84d4"
    node_data = node[1] # Dictionary with type, utterances, timestamp, etc.

    if node_data['utterances'][0] in parent_actions: # If the node is a parent action node

        real_utterance = node_data['utterances'][0] 
        predicted_utterances = [] 

        # Iterate through all the sub-actions and make predictions using baseline_dict.
        for u, v, data in test.edges(node_id, data=True):
            if data.get('link_type') == 'has_element':
                sub_action = test.nodes[v].get('utterances')[0]
                
                # If sub action has object, keep it. 
                for a, b, link in test.out_edges(v, data=True):
                    if link['link_type'] == 'has_object':
                        sub_action = sub_action + ' ' + test.nodes[b].get('utterances')[0]
                        # As a result, sub action will look like: stir whisk, cut knife, etc...

                prediction = baseline_dict.get(sub_action)
                predicted_utterances.append(prediction)
                    
        # Keep top 1 prediction
        predicted_utterance = max(predicted_utterances,key=predicted_utterances.count) 
        predictions.append(predicted_utterance)
        # Keep the real value
        real_utterances.append(real_utterance)
        # Keep all the predictions with corresponding votes
        predictions_probab.append({i:predicted_utterances.count(i) for i in set(predicted_utterances)})
     

### Evaluate Baseline

In [19]:
def hits_at_one(true, pred):
    # true and pred are lists

    n = len(true)
    tp = sum([int(true[i]==pred[i]) for i in range (n)])
    return tp/n

def hits_at_k(true, pred, k):
    # true is a list, pred is a list of dictionaries, 
    # e.g. {'task_5_k_cereals': 8, 'task_8_w_hammering': 4, 'task_9_w_sawing': 1}

    n = len(true)
    hits = np.zeros(n)
    
    for p in range (1, k+1):
        # get key with p-th larest value from each dictionary in the pred list
        pred_i = [sorted(pred[i], key=pred[i].get)[-p] if p <= len(pred[i]) else 'None' for i in range (n)]
        for hit in range (n):
            if hits[hit] == 0 and pred_i[hit] == true[hit]:
                hits[hit] = 1
        
    tp = sum(hits)
    return tp/n

In [20]:
hits_at_one(real_utterances, predictions)

0.75

In [21]:
for k in range (1, 4):
    print(f'Hits@{k}: ', hits_at_k(real_utterances, predictions_probab, k))

Hits@1:  0.7592592592592593
Hits@2:  0.9722222222222222
Hits@3:  1.0


In [22]:
confusion_matrix(real_utterances, predictions)

array([[12,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 3,  9,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0, 12,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  9,  3,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, 12,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  9,  3,  0,  0],
       [ 0,  0,  0,  0,  0,  3,  9,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 12,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  9,  3]], dtype=int64)